In [1]:
from bs4 import BeautifulSoup
import requests

competition_urls = {
	'football': 
	{
		"ligue1": "https://www.netbet.fr/football/france/ligue-1-uber-eats",
		"liga": "https://www.netbet.fr/football/espagne/laliga",
		"bundesliga": "https://www.netbet.fr/football/allemagne/bundesliga",
		"premier-league": "https://www.netbet.fr/football/angleterre/premier-league",
		"serie-a": "https://www.netbet.fr/football/italie/coupe-d-italie",
		"primeira": "https://www.netbet.fr/football/portugal/primeira-liga",
		"serie-a-brasil": "https://www.netbet.fr/football/bresil/brasileirao",
		"a-league": "https://www.netbet.fr/football/australie/a-league",
		"bundesliga-austria": "https://www.netbet.fr/football/autriche/bundesliga",
		"division-1a": "https://www.netbet.fr/football/belgique/pro-league",
		"super-lig": "https://www.netbet.fr/football/turquie/super-lig",
	},
	'basketball':
	{
		"nba": "https://www.netbet.fr/basketball/etats-unis/nba",
		"euroleague": "https://www.netbet.fr/basketball/coupes-d-europe/euroligue",
	}
}

def get_page(competition):
	if (competition["sport"] in competition_urls and competition["competition"] in competition_urls[competition["sport"]]):
		url = competition_urls[competition["sport"]][competition["competition"]]
	else:
		return None
	response = requests.get(url, headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36"})
	html = BeautifulSoup(response.content, 'html.parser')
	return html

def get_games(competition):
	html = get_page(competition)
	games = []
	game_elements = html.select(".nb-event")
	for el in game_elements:
		names = el.select(".nb-match_actor")
		team1 = "".join(names[0].text.split())
		team2 = "".join(names[1].text.split())
		odd_els = el.select(".nb-odds_amount")
		odds = []
		for odd_el in odd_els[:3]:
			odds.append(float(odd_el.text.replace(",", ".")))
		games.append({
			'team1': team1,
			'team2': team2,
			'odds': odds
		})
	return games

In [3]:
# maintenant on souhaite manipuler des dataframes : 
import pandas as pd 
import numpy as np 
def into_a_pd(competition):
    games = get_games(competition=competition)
    team1 = []
    team2 = []
    odd1 = []
    odd2 = []
    oddnul = []
    if len(games[0]['odds']) == 3 :
        for i in range(len(games)) :
            team1.append(games[i]['team1'])
            team2.append(games[i]['team2'])
            odd1.append(games[i]['odds'][0])
            oddnul.append(games[i]['odds'][1])
            odd2.append(games[i]['odds'][2]) 

            data = {
                'team1' : team1,
                'team2' : team2,
                'odd of team 1 winning' : odd1,
                'odds of equality ': oddnul,
                'odd of team 2 winning' : odd2,
            }
    else :
        for i in range(len(games)) :
            team1.append(games[i]['team1'])
            team2.append(games[i]['team2'])
            odd1.append(games[i]['odds'][0])
            odd2.append(games[i]['odds'][1])
        data = {
                'team1' : team1,
                'team2' : team2,
                'odd of team 1 winning' : odd1,
                'odd of team 2 winning' : odd2,
            }
   

    df = pd.DataFrame(data=data)

    return df 

into_a_pd(competition = {'sport': 'football', 'competition': 'ligue1'})

,team1,team2,odd of team 1 winning,odds of equality,odd of team 2 winning
0,Lorient,Montpellier,20.00,5.00,1.09
1,Reims,Rennes,3.40,3.25,2.00
2,Marseille,Toulouse,1.35,4.90,6.75
3,Nice,Lens,2.92,2.95,2.42
4,Monaco,Brest,1.40,4.70,5.90
5,Toulouse,ACAjaccio,1.97,3.20,3.60
6,Angers,Lorient,2.72,3.15,2.42
7,FCNantes,Auxerre,1.85,3.20,4.00
8,Lyon,Clermont,1.34,4.80,7.00
9,Lens,PSG,3.10,3.45,2.05
